****
# Multiprocessing MonteCarlo Simulation for Option Pricing
****

## About this notebook: 
Notebook prepared by **Jesus Perez Colino** Version 0.1, First Released: 01/12/2014, Alpha.  

- This work is licensed under a [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). This work is offered for free, with the hope that it will be useful.


- **Summary**: This notebook is just the simplest implementation of a Multiprocessing Monte-Carlo Simulation engine for the **European option** pricing.


- **Python & packages versions** to reproduce the results of this notebook: 

In [2]:
import numpy as np
import IPython
from scipy.stats import norm
from abc import ABCMeta, abstractmethod
from sys import version 
import multiprocessing
from numpy import ceil, mean
import time
import os

print ' Reproducibility conditions for this notebook '.center(85,'-')
print 'Python version:     ' + version
print 'Numpy version:      ' + np.__version__
print 'IPython version:    ' + IPython.__version__
print '-'*85

-------------------- Reproducibility conditions for this notebook -------------------
Python version:     2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Sep 15 2015, 14:29:08) 
[GCC 4.2.1 (Apple Inc. build 5577)]
Numpy version:      1.9.2
IPython version:    4.0.0
-------------------------------------------------------------------------------------


## Base and Derived Classes for the Option Pricing

In [3]:
class EuropeanOption(object):
    """ Abstract Class for European options. Partially implemented.
    S0 : float : initial stock/index level
    strike : float : strike price
    T : float : time to maturity (in year fractions)
    r : float : constant risk-free short rate
    div :    float : dividend yield
    sigma :  float : volatility factor in diffusion term
    model: str: name of the model for the pricing"""

    __metaclass__ = ABCMeta

    def __init__(self, option_type, S0, strike, T, r, div, sigma, model):
        try:
            self.option_type = option_type
            assert isinstance(option_type, str)
            self.S0 = float(S0)
            self.strike = float(strike)
            self.T = float(T)
            self.r = float(r)
            self.div = float(div)
            self.sigma = float(sigma)
            self.model = str(model)
        except ValueError:
            print('Error passing Options parameters')

        models = ['BlackScholes', 'MonteCarlo', 
                  'BinomialTree', 'TrinomialTree',
                  'FFT', 'PDE']
        
        if model not in models:
            raise Exception('Error: Model unknown')
            
        option_types = ['call', 'put']
        
        if option_type not in option_types:
            raise ValueError("Error: Option type not valid. Enter 'call' or 'put'")
        if S0 < 0 or strike < 0 or T <= 0 or r < 0 or div < 0 or sigma < 0:
            raise ValueError('Error: Negative inputs not allowed')
            
        self.discount = np.exp(-self.r * self.T)

    def getmodel(self):
        return self.model

    def __str__(self):
        return "This European Option is priced using {0}".format(self.getmodel())

    @abstractmethod
    def value(self):
        pass
    
    @abstractmethod
    def delta(self):
        pass
        

In [33]:
class BlackScholes(EuropeanOption):

    def __init__(self, option_type, S0, strike, T, r, div, sigma):
        EuropeanOption.__init__(self,option_type, S0, strike, 
                                T, r, div, sigma, 'BlackScholes')
        
        d1 = ((np.log(self.S0 / self.strike) + 
              (self.r - self.div + 0.5 * (self.sigma ** 2)) * self.T) / 
              float( self.sigma * np.sqrt(self.T)))
        d2 = float(d1 - self.sigma * np.sqrt(self.T))
        self.Nd1 = norm.cdf(d1, 0, 1)
        self.Nnd1 = norm.cdf(-d1, 0, 1)
        self.Nd2 = norm.cdf(d2, 0, 1)
        self.Nnd2 = norm.cdf(-d2, 0, 1)
        self.pNd1 = norm.pdf(d1, 0, 1)
        
    @property
    def value(self):
        if self.option_type == 'call':
            value = (self.S0 * np.exp(-self.div * self.T) * self.Nd1 -
                     self.strike * np.exp(-self.r * self.T) * self.Nd2)
        else:
            value = (self.strike * np.exp(-self.r * self.T) * self.Nnd2 -
                     self.S0 * np.exp(-self.div * self.T) * self.Nnd1)
        return value
    
    @property
    def delta(self):
        if self.option_type == 'call':
            delta = np.exp(- self.div * self.T) * self.Nd1
        else:
            delta = np.exp(- self.div * self.T) * (self.Nd1 - 1)
        return delta

In [34]:
class MonteCarlo(EuropeanOption):

    def __init__(self, option_type, S0, strike, T, r, div, sigma, 
                 simulations = 500000, 
                 antithetic = True, 
                 moment_matching = True, 
                 fixed_seed = True):
        EuropeanOption.__init__(self, option_type, S0, strike, T, r, div, sigma, "MonteCarlo")
        self.simulations = int(simulations)
        self.antithetic = bool(antithetic)
        self.moment_matching = bool(moment_matching)
        self.fixed_seed = bool(fixed_seed)
        try:
            if self.simulations > 0 :
                assert isinstance(self.simulations, int)
        except:
            raise ValueError("Simulation's number has to be positive integer")
    
    def simulation_terminal(self, seed = 1234567890):
        if self.fixed_seed:
            assert isinstance(seed, int)
            np.random.seed(seed)
        if self.antithetic: 
            brownian = np.random.standard_normal(size = int(np.ceil(self.simulations/2.)))
            brownian = np.concatenate((brownian, -brownian))
        else:
            brownian = np.random.standard_normal(size = self.simulations)
        if self.moment_matching: 
            brownian = brownian - np.mean(brownian)
            brownian = brownian / np.std(brownian)
            
        price_terminal = self.S0 * np.exp((self.r - self.div - 0.5 * self.sigma ** 2) *
                                          self.T +
                                          self.sigma * np.sqrt(self.T) * brownian)
        return price_terminal

    def generate_payoffs(self):
        price_terminal = self.simulation_terminal()
        if self.option_type == 'call':
            payoff = np.maximum((price_terminal - self.strike), 0)
        else:
            payoff = np.maximum((self.strike - price_terminal), 0)
        return payoff

    @property
    def value(self):
        payoff = self.generate_payoffs()
        return self.discount * np.sum(payoff) / float(len(payoff))
    
    @property
    def delta(self):
        diff = self.S0 * 0.01
        myCall_1 = MonteCarlo(self.option_type, self.S0 + diff, 
                              self.strike, self.T, self.r, self.div, self.sigma)
        myCall_2 = MonteCarlo(self.option_type, self.S0 - diff, 
                              self.strike, self.T, self.r, self.div, self.sigma)
        return (myCall_1.value - myCall_2.value) / float(2. * diff)

## Multiprocessing MonteCarlo: Functions, Pool, Mapping and Join

In [35]:
def eval_in_pool(simulations, method = 'value'):
    try:
        assert isinstance(simulations, int)
        if method is not None:
            assert isinstance(method, str)
    except ValueError:
        print 'Type in the parameters incorrect'
    myCall = MonteCarlo('call', 100., 100., .5, 0.01, 0., .35, simulations)
    def eval_value(simulations):
        return myCall.value
    def eval_delta(simulations):
        return myCall.delta
    if method == 'value':
        return eval_value(simulations)
    if method == 'delta':
        return eval_delta(simulations)
            

In [36]:

c = BlackScholes('call', 100., 100., .5, 0.01, 0., .35)
print 'BS Price:', c.value
print '-' * 75
scenarios = {'1': [1e4, 5e7], 
             '2': [1e4, 5e7], 
             '3': [1e4, 5e7],
             '4': [1e4, 5e7],
             '5': [1e4, 5e7],
             '6': [1e4, 5e7]}
for num_processes in scenarios:
    for N in scenarios[num_processes]:
        start = time.time()
        chunks = [int(ceil(N / int(num_processes)))] * int(num_processes)
        chunks[-1] = int(chunks[-1] - sum(chunks) + N)
        p = multiprocessing.Pool(int(num_processes))
        option_price = p.map(eval_in_pool, chunks)
        p.close()
        p.join()
        end = time.time()
        print 'Number of processors:', num_processes + ',',
        print 'Number of simulations:', str(int(N))
        print 'Monte Carlo Option Price:', str(mean(option_price)) + ',',
        print 'Time, in sec:', str(end - start)
        print '-' * 75

BS Price: 10.074989532
---------------------------------------------------------------------------
Number of processors: 1, Number of simulations: 10000
Monte Carlo Option Price: 10.0555595453, Time, in sec: 0.115061998367
---------------------------------------------------------------------------
Number of processors: 1, Number of simulations: 50000000
Monte Carlo Option Price: 10.0746810935, Time, in sec: 4.16948795319
---------------------------------------------------------------------------
Number of processors: 3, Number of simulations: 10000
Monte Carlo Option Price: 10.0701067309, Time, in sec: 0.123365879059
---------------------------------------------------------------------------
Number of processors: 3, Number of simulations: 50000000
Monte Carlo Option Price: 10.074778969, Time, in sec: 1.40977096558
---------------------------------------------------------------------------
Number of processors: 2, Number of simulations: 10000
Monte Carlo Option Price: 10.0494614158, Tim